<a href="https://colab.research.google.com/github/aryanshu/Delhi_Election_poll_via_sentiment_analysis_using_tweepy/blob/master/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
cd '/gdrive/My Drive/Colab Notebooks'

/gdrive/My Drive/Colab Notebooks


In [0]:
!pip install tweepy
!pip install textblob
!pip install jsonpickle

In [0]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import Stream

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import time

API_KEY='MuTZu584Tok038SCEojrjrKRe'
API_SECRET='fsBbryvLU1pKRxXYGtKbuW5v7qZEMBVYXmzvT8aEqnbTKo7mgj'
auth = tweepy.AppAuthHandler(API_KEY, API_SECRET)

api = tweepy.API(auth,wait_on_rate_limit=True,
				   wait_on_rate_limit_notify=True)


In [0]:
class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1

    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text[:150] for tweet in tweets], columns=['tweets'])

        df['id'] = np.array([tweet.id for tweet in tweets])
        df['date'] = np.array([tweet.created_at for tweet in tweets])
 
        return df

In [0]:
import sys
import jsonpickle
import os

print("Downloading max {0} tweets".format(maxTweets))
def tweet_collect(query,maxTweets=1000,fName='tweets.txt',until=None):
  sinceId = None
  tweets=[]
  max_id =-1000000

  tweetCount = 0
  searchQuery = query  
  tweetsPerQry = 100 
  with open(fName, 'w') as f:
      while tweetCount < maxTweets:
          try:
              if (max_id <= 0):
                  if (not sinceId):
                      new_tweets = api.search(q=searchQuery, count=tweetsPerQry,lang="en",wait_on_rate_limit=True,
                                geocode='28.644800,77.216721,20km',until=until)
                  else:
                      new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                              since_id=sinceId,lang="en",wait_on_rate_limit=True,
                                geocode='28.644800,77.216721,20km',until=until)
              else:
                  if (not sinceId):
                      new_tweets = api.search(q=searchQuery, count=tweetsPerQry,lang="en",wait_on_rate_limit=True,
                                geocode='28.644800,77.216721,20km',until=until,
                                              max_id=str(max_id - 1))
                  else:
                      new_tweets = api.search(q=searchQuery, count=tweetsPerQry,lang="en",wait_on_rate_limit=True,
                                geocode='28.644800,77.216721,20km',until=until,
                                              max_id=str(max_id - 1),
                                              since_id=sinceId)
              if not new_tweets:
                  print("No more tweets found")
                  break
              for tweet in new_tweets:
                  f.write(jsonpickle.encode(tweet._json, unpicklable=False) +
                          '\n')
                  tweets.append(tweet)
              tweetCount += len(new_tweets)
              print("Downloaded {0} tweets".format(tweetCount))
              max_id = new_tweets[-1].id
          except tweepy.TweepError as e:
              # Just exit if any error
              print("some error : " + str(e))
              break

  print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))
  return tweets


In [0]:
tweets=[]
for i in range(1,9):
  until='2020-2-'+str(i)
  tweet_on=tweet_collect(query='AAP',maxTweets=20000,until=until)
  tweets=tweets+tweet_on

tweets2=[]
for i in range(1,9):
  until='2020-2-'+str(i)
  tweet_on2=tweet_collect(query='BJP',maxTweets=20000,until=until)
  tweets2=tweets2+tweet_on2
  
tweets3=[]
for i in range(1,9):
  until='2020-2-'+str(i)
  tweet_on=tweet_collect(query='CONGRESS',maxTweets=10000,until=until)
  tweets3=tweets3+tweet_on

In [0]:
df.to_csv('AAP_tweet.csv')
df2.to_csv('BJP_tweet.csv')
df3.to_csv('CONGRESS_tweet.csv')
df.date

0       2020-02-04 23:59:58
1       2020-02-04 23:59:48
2       2020-02-04 23:59:46
3       2020-02-04 23:59:45
4       2020-02-04 23:59:31
                ...        
38013   2020-02-07 10:35:49
38014   2020-02-07 10:35:49
38015   2020-02-07 10:35:47
38016   2020-02-07 10:35:46
38017   2020-02-07 10:35:36
Name: date, Length: 38018, dtype: datetime64[ns]

In [0]:
df = tweet_analyzer.tweets_to_data_frame(tweets)
print(len(df))
df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])
aap_positive=100*len(df['sentiment'][df['sentiment']==1])/len(df)
aap_negative=100*len(df['sentiment'][df['sentiment']==-1])/len(df)
t_score=aap_positive+aap_negative
aap_score=(aap_positive)/aap_negative

print('AAP positive {}%'.format(100*len(df['sentiment'][df['sentiment']==1])/len(df)))
print('AAP negative {}%'.format(100*len(df['sentiment'][df['sentiment']==-1])/len(df)))
print('AAP neutral {}%'.format(100*len(df['sentiment'][df['sentiment']==0])/len(df)))



df2 = tweet_analyzer.tweets_to_data_frame(tweets2)
print(len(df2))
df2['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df2['tweets']])
bjp_positive=100*len(df2['sentiment'][df2['sentiment']==1])/len(df2)
bjp_negative=100*len(df2['sentiment'][df2['sentiment']==-1])/len(df2)
t2_score=bjp_positive+bjp_negative
bjp_score=(bjp_positive)/(bjp_negative)


print('BJP positive {}%'.format(100*len(df2['sentiment'][df2['sentiment']==1])/len(df2)))
print('BJP negative {}%'.format(100*len(df2['sentiment'][df2['sentiment']==-1])/len(df2)))
print('BJP neutral {}%'.format(100*len(df2['sentiment'][df2['sentiment']==0])/len(df2)))

print('AAP Adjusted Voting Percentage  {}'.format(92.5*aap_score/(aap_score+bjp_score)))
print('BJP Adjusted Voting Percentage  {}'.format(92.5*bjp_score/(aap_score+bjp_score)))



df3 = tweet_analyzer.tweets_to_data_frame(tweets3)
print(len(df3))
df3['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df3['tweets']])
cong_positive=100*len(df3['sentiment'][df3['sentiment']==1])/len(df3)
cong_negative=100*len(df3['sentiment'][df3['sentiment']==-1])/len(df3)
t_score=cong_positive+cong_negative
cong_score=(cong_positive)/cong_negative

print('CONG positive {}%'.format(100*len(df3['sentiment'][df3['sentiment']==1])/len(df3)))
print('CONG negative {}%'.format(100*len(df3['sentiment'][df3['sentiment']==-1])/len(df3)))
print('CONG neutral {}%'.format(100*len(df3['sentiment'][df3['sentiment']==0])/len(df3)))

print('time: {} '.format((end-start)))

80347
AAP positive 29.509502532764134%
AAP negative 18.539584552005675%
AAP neutral 51.95091291523019%
57890
BJP positive 33.70012091898428%
BJP negative 33.10934531007082%
BJP neutral 33.190533770944896%
AAP Adjusted Voting Percentage  56.42073330727944
BJP Adjusted Voting Percentage  36.07926669272056
time: 780.7540986537933 


In [0]:
print('AAP Adjusted Voting Percentage  {}'.format(100*(aap_score )/((aap_score)+bjp_score+cong_score)))
print('BJP Adjusted Voting Percentage  {}'.format(100*(bjp_score)/(aap_score+(bjp_score)+cong_score)))
print('cong Adjusted Voting Percentage  {}'.format(100*(cong_score)/(aap_score+bjp_score+(cong_score))))

AAP Adjusted Voting Percentage  44.732836556
BJP Adjusted Voting Percentage  29.758831419163865
cong Adjusted Voting Percentage  25.50833202483613
